In [ ]:
# Enhanced image embedding extraction script with full bird metadata
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pickle
import os
from tqdm import tqdm

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load pre-trained ResNet50 model
# Option 1: Use standard ImageNet pre-trained ResNet
model = models.resnet50(pretrained=True)
# Remove the final classification layer to get feature embeddings
model = nn.Sequential(*list(model.children())[:-1])  # This gives 2048D features
model.eval()
model.to(device)

# Alternative: You could use a model specifically trained on birds like:
# from transformers import AutoImageProcessor, AutoModel
# processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
# model = AutoModel.from_pretrained("google/vit-base-patch16-224")

# Image preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your bird images metadata
# Assuming your CSV data is in a file called 'bird_images_metadata.csv'

# If you have the CSV data in a file
images_df = pd.read_csv("wikimedia_bird_images.csv")
    
print(f"Loaded {len(images_df)} image records")
print(f"Unique species: {images_df['species_name'].nunique()}")
print(f"Unique bird_ids: {images_df['bird_id'].nunique()}")

features_list = []
failed_images = []

# Process each image
for idx, row in tqdm(images_df.iterrows(), total=len(images_df), desc="Processing images"):
    print(f"Processing bird_id {row['bird_id']}: {row['species_name']} (image {row['image_number']})")
    
    try:
        # Load and preprocess image
        image_path = row['image_path']
        
        # Handle different path formats (Windows vs Unix)
        if not os.path.exists(image_path):
            # Try with forward slashes
            image_path = image_path.replace('\\', '/')
        
        if not os.path.exists(image_path):
            print(f"⚠️  Image not found: {image_path}")
            failed_images.append(row['image_path'])
            continue
            
        # Load and transform image
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image).unsqueeze(0).to(device)
        
        # Extract features
        with torch.no_grad():
            features = model(image_tensor)
            features = features.view(features.size(0), -1)  # Flatten
            features = features.squeeze().cpu().numpy()
        
        # Save ALL metadata including bird_id
        features_list.append({
            'bird_id': int(row['bird_id']),              # Important: Bird ID
            'species_name': row['species_name'],          # Species name
            'image_path': row['image_path'],              # Original image path
            'image_number': int(row['image_number']),     # Image number for this species
            'source_url': row['source_url'],              # Original source URL
            'source_title': row['source_title'],          # Source title
            'width': int(row['width']),                   # Image dimensions
            'height': int(row['height']),
            'file_size_mb': float(row['file_size_mb']),   # File size
            'source': row['source'],                      # Source (Wikimedia Commons)
            'license': row['license'],                    # License info
            'quality_score': int(row['quality_score']),   # Quality rating
            'features': features,                         # 2048D ResNet features
            'feature_shape': features.shape,              # Vector shape info
            'model_used': 'ResNet50_ImageNet'             # Model identifier
        })
        
    except Exception as e:
        print(f"❌ Error processing {row['image_path']}: {str(e)}")
        failed_images.append(row['image_path'])
        continue

# Save features with all metadata
output_file = 'bird_image_features.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(features_list, f)

print(f"\n✅ Extracted {len(features_list)} image feature vectors with bird_ids!")
print(f"💾 Saved to: {output_file}")

if failed_images:
    print(f"⚠️  Failed to process {len(failed_images)} images:")
    for failed in failed_images[:10]:  # Show first 10 failures
        print(f"   - {failed}")
    if len(failed_images) > 10:
        print(f"   ... and {len(failed_images) - 10} more")

# Quick verification and statistics
print(f"\n📊 Processing Summary:")
print(f"   - Total images processed: {len(features_list)}")
print(f"   - Feature vector size: {features_list[0]['feature_shape'] if features_list else 'N/A'}")
print(f"   - Unique bird_ids: {len(set(item['bird_id'] for item in features_list))}")
print(f"   - Unique species: {len(set(item['species_name'] for item in features_list))}")

# Show sample of bird IDs saved
if features_list:
    sample_bird_ids = [item['bird_id'] for item in features_list[:5]]
    sample_species = [item['species_name'] for item in features_list[:5]]
    print(f"   - Sample bird_ids: {sample_bird_ids}")
    print(f"   - Sample species: {sample_species}")

# Save a summary CSV for reference
summary_df = pd.DataFrame([
    {
        'bird_id': item['bird_id'],
        'species_name': item['species_name'],
        'image_number': item['image_number'],
        'feature_shape': str(item['feature_shape']),
        'quality_score': item['quality_score']
    }
    for item in features_list
])

summary_df.to_csv('bird_image_features_summary.csv', index=False)
print(f"📋 Feature summary saved to: bird_image_features_summary.csv")

Using device: cpu


d:\Qdrant\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Qdrant\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded 423 image records
Unique species: 88
Unique bird_ids: 88


Processing images:   0%|          | 0/423 [00:00<?, ?it/s]

Processing bird_id 0: African Pied Wagtail (image 1)


...

Processing bird_id 87: Yellowhammer (image 2)
Processing bird_id 87: Yellowhammer (image 3)


Processing images: 100%|█████████▉| 422/423 [02:44<00:00,  4.97it/s]

Processing bird_id 87: Yellowhammer (image 4)
Processing bird_id 87: Yellowhammer (image 5)


Processing images: 100%|██████████| 423/423 [02:44<00:00,  2.57it/s]


✅ Extracted 423 image feature vectors with bird_ids!
💾 Saved to: bird_image_features_complete.pkl

📊 Processing Summary:
   - Total images processed: 423
   - Feature vector size: (2048,)
   - Unique bird_ids: 88
   - Unique species: 88
   - Sample bird_ids: [0, 0, 0, 0, 0]
   - Sample species: ['African Pied Wagtail', 'African Pied Wagtail', 'African Pied Wagtail', 'African Pied Wagtail', 'African Pied Wagtail']
📋 Feature summary saved to: bird_image_features_summary.csv


In [ ]:
# Simple script to check extracted bird image features - statistics only
import pickle
import numpy as np
from collections import Counter

def check_image_features(pickle_file='bird_image_features.pkl'):
    """Load and display basic statistics for image features"""
    
    print("🔍 Loading image features...")
    with open(pickle_file, 'rb') as f:
        features_list = pickle.load(f)
    
    print(f"✅ Loaded {len(features_list)} image feature records\n")
    
    # Basic info
    print("📋 BASIC INFO")
    print("=" * 40)
    first_item = features_list[0]
    print(f"Feature vector shape: {first_item['features'].shape}")
    print(f"Feature vector dtype: {first_item['features'].dtype}")
    print(f"Model used: {first_item.get('model_used', 'Unknown')}")
    
    # Extract all features and metadata
    all_features = np.array([item['features'] for item in features_list])
    bird_ids = [item['bird_id'] for item in features_list]
    species_names = [item['species_name'] for item in features_list]
    quality_scores = [item['quality_score'] for item in features_list]
    
    print(f"\n📊 FEATURE STATISTICS")
    print("=" * 40)
    print(f"Total images: {len(features_list)}")
    print(f"Feature matrix shape: {all_features.shape}")
    print(f"Memory usage: {all_features.nbytes / 1024 / 1024:.2f} MB")
    
    print(f"\nFeature values:")
    print(f"  Mean: {np.mean(all_features):.6f}")
    print(f"  Std:  {np.std(all_features):.6f}")
    print(f"  Min:  {np.min(all_features):.6f}")
    print(f"  Max:  {np.max(all_features):.6f}")
    
    # Data quality
    nan_count = np.isnan(all_features).sum()
    inf_count = np.isinf(all_features).sum()
    zero_vectors = np.sum(np.all(all_features == 0, axis=1))
    
    print(f"\nData quality:")
    print(f"  NaN values: {nan_count}")
    print(f"  Infinite values: {inf_count}")
    print(f"  Zero vectors: {zero_vectors}")
    
    # Bird data stats
    print(f"\n🐦 BIRD DATA")
    print("=" * 40)
    unique_bird_ids = len(set(bird_ids))
    unique_species = len(set(species_names))
    
    print(f"Unique bird IDs: {unique_bird_ids}")
    print(f"Unique species: {unique_species}")
    
    # Images per bird_id
    bird_counts = Counter(bird_ids)
    print(f"\nImages per bird_id:")
    print(f"  Min: {min(bird_counts.values())}")
    print(f"  Max: {max(bird_counts.values())}")
    print(f"  Mean: {np.mean(list(bird_counts.values())):.1f}")
    
    # Images per species
    species_counts = Counter(species_names)
    print(f"\nImages per species:")
    print(f"  Min: {min(species_counts.values())}")
    print(f"  Max: {max(species_counts.values())}")
    print(f"  Mean: {np.mean(list(species_counts.values())):.1f}")
    
    # Quality scores
    quality_dist = Counter(quality_scores)
    print(f"\nQuality score distribution:")
    for score in sorted(quality_dist.keys()):
        print(f"  Score {score}: {quality_dist[score]} images")
    
    # Sample data preview
    print(f"\n📝 SAMPLE DATA")
    print("=" * 40)
    print("First 3 records:")
    for i in range(min(3, len(features_list))):
        item = features_list[i]
        print(f"  {i+1}. Bird ID {item['bird_id']}: {item['species_name']}")
        print(f"     Image {item['image_number']}, Quality: {item['quality_score']}")
        print(f"     Feature norm: {np.linalg.norm(item['features']):.3f}")
    
    print(f"\n✅ Feature check complete!")
    
    return {
        'total_images': len(features_list),
        'feature_shape': all_features.shape,
        'unique_birds': unique_bird_ids,
        'unique_species': unique_species,
        'feature_stats': {
            'mean': np.mean(all_features),
            'std': np.std(all_features),
            'min': np.min(all_features),
            'max': np.max(all_features)
        },
        'data_quality': {
            'nan_count': nan_count,
            'inf_count': inf_count,
            'zero_vectors': zero_vectors
        }
    }

if __name__ == "__main__":
    # Run the check
    stats = check_image_features()

🔍 Loading image features...
✅ Loaded 423 image feature records

📋 BASIC INFO
Feature vector shape: (2048,)
Feature vector dtype: float32
Model used: ResNet50_ImageNet

📊 FEATURE STATISTICS
Total images: 423
Feature matrix shape: (423, 2048)
Memory usage: 3.30 MB

Feature values:
  Mean: 0.439698
  Std:  0.422492
  Min:  0.000000
  Max:  9.125230

Data quality:
  NaN values: 0
  Infinite values: 0
  Zero vectors: 0

🐦 BIRD DATA
Unique bird IDs: 88
Unique species: 88

Images per bird_id:
  Min: 2
  Max: 5
  Mean: 4.8

Images per species:
  Min: 2
  Max: 5
  Mean: 4.8

Quality score distribution:
  Score 3: 2 images
  Score 4: 18 images
  Score 5: 54 images
  Score 6: 349 images

📝 SAMPLE DATA
First 3 records:
  1. Bird ID 0: African Pied Wagtail
     Image 1, Quality: 6
     Feature norm: 22.449
  2. Bird ID 0: African Pied Wagtail
     Image 2, Quality: 6
     Feature norm: 25.111
  3. Bird ID 0: African Pied Wagtail
     Image 3, Quality: 6
     Feature norm: 23.488

✅ Feature check co

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import pickle
import os
from dotenv import load_dotenv
import numpy as np

# Load environment variables
load_dotenv()

def upload_image_features():
    """Upload bird image features to Qdrant Cloud"""
    
    # Load features from the features folder
    features_file = 'features/bird_image_features.pkl'
    
    print(f"Loading image features from {features_file}...")
    
    try:
        with open(features_file, 'rb') as f:
            features_list = pickle.load(f)
    except FileNotFoundError:
        print(f"File not found: {features_file}")
        print("Available files in features folder:")
        if os.path.exists('features'):
            for file in os.listdir('features'):
                if file.endswith('.pkl'):
                    print(f"  - {file}")
        return False
    
    print(f"Loaded {len(features_list)} image records")
    
    # Connect to Qdrant Cloud
    client = QdrantClient(
        url=os.getenv('QDRANT_ENDPOINT'),
        api_key=os.getenv('QDRANT_API_KEY'),
    )
    
    print(f"Connected to Qdrant Cloud: {os.getenv('QDRANT_ENDPOINT')}")
    
    # Get feature dimensions
    sample_features = features_list[0]['features']
    feature_dim = len(sample_features)
    print(f"Feature dimensions: {feature_dim}")
    
    # Create collection
    collection_name = "bird_image_search"
    
    try:
        # Delete collection if it exists
        try:
            client.delete_collection(collection_name)
            print(f"Deleted existing collection: {collection_name}")
        except:
            pass
        
        # Create new collection
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=feature_dim, distance=Distance.COSINE)
        )
        print(f"Created collection: {collection_name}")
        
    except Exception as e:
        print(f"Error creating collection: {e}")
        return False
    
    # Prepare points for upload
    points = []
    
    for idx, item in enumerate(features_list):
        # Create point with comprehensive metadata
        point = PointStruct(
            id=idx,  # Use index as point ID
            vector=item['features'].tolist() if isinstance(item['features'], np.ndarray) else item['features'],
            payload={
                "bird_id": item['bird_id'],
                "species_name": item['species_name'],
                "image_path": item['image_path'],
                "image_number": item['image_number'],
                "source_url": item['source_url'],
                "source_title": item['source_title'],
                "width": item['width'],
                "height": item['height'],
                "file_size_mb": item['file_size_mb'],
                "source": item['source'],
                "license": item['license'],
                "quality_score": item['quality_score'],
                "model_used": item['model_used'],
                "feature_type": "resnet50_imagenet"
            }
        )
        points.append(point)
    
    # Upload to Qdrant in batches
    batch_size = 100
    total_batches = (len(points) + batch_size - 1) // batch_size
    
    print(f"Uploading {len(points)} points in {total_batches} batches...")
    
    try:
        for i in range(0, len(points), batch_size):
            batch = points[i:i + batch_size]
            client.upsert(collection_name=collection_name, points=batch)
            print(f"Uploaded batch {i//batch_size + 1}/{total_batches} ({len(batch)} points)")
        
        print(f"Successfully uploaded {len(points)} image features to Qdrant Cloud!")
        
    except Exception as e:
        print(f"Error uploading: {e}")
        return False
    
    # Test search functionality
    print("\nTesting search functionality...")
    
    try:
        # Get a sample vector for testing
        test_vector = features_list[0]['features'].tolist() if isinstance(features_list[0]['features'], np.ndarray) else features_list[0]['features']
        test_bird = features_list[0]
        
        results = client.search(
            collection_name=collection_name,
            query_vector=test_vector,
            limit=5
        )
        
        print(f"\nSimilar images to {test_bird['species_name']} (bird_id: {test_bird['bird_id']}):")
        for result in results:
            print(f"  ID {result.id}: {result.payload['species_name']} (bird_id: {result.payload['bird_id']}, score: {result.score:.3f})")
        
    except Exception as e:
        print(f"Error testing search: {e}")
    
    # Collection summary
    try:
        collection_info = client.get_collection(collection_name)
        print(f"\nCollection Summary:")
        print(f"  Name: {collection_name}")
        print(f"  Points: {collection_info.points_count}")
        print(f"  Dimensions: {collection_info.config.params.vectors.size}")
        print(f"  Status: {collection_info.status}")
        
    except Exception as e:
        print(f"Error getting collection info: {e}")
    
    return True

if __name__ == "__main__":
    print("Bird Image Features Uploader for Qdrant Cloud")
    print("=" * 50)
    
    # Upload features
    success = upload_image_features()
    
    if success:
        print("\nImage features successfully uploaded!")
        
    else:
        print("Upload failed. Please check your files and credentials.")

Bird Image Features Uploader for Qdrant Cloud
Loading image features from features/bird_image_features.pkl...
Loaded 423 image records
Connected to Qdrant Cloud: https://559d3608-530f-4a96-97f3-4bb6202a5fb9.us-west-1-0.aws.cloud.qdrant.io
Feature dimensions: 2048
Deleted existing collection: bird_image_search
Created collection: bird_image_search
Uploading 423 points in 5 batches...
Uploaded batch 1/5 (100 points)
Uploaded batch 2/5 (100 points)
Uploaded batch 3/5 (100 points)
Uploaded batch 4/5 (100 points)
Uploaded batch 5/5 (23 points)
Successfully uploaded 423 image features to Qdrant Cloud!

Testing search functionality...

Similar images to African Pied Wagtail (bird_id: 0):
  ID 0: African Pied Wagtail (bird_id: 0, score: 1.000)
  ID 4: African Pied Wagtail (bird_id: 0, score: 0.879)
  ID 169: Eurasian Jay (bird_id: 34, score: 0.851)
  ID 3: African Pied Wagtail (bird_id: 0, score: 0.844)
  ID 167: Eurasian Jay (bird_id: 34, score: 0.830)

Collection Summary:
  Name: bird_image_

C:\Users\parth\AppData\Local\Temp\ipykernel_16568\416931229.py:121: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [ ]:
def search_similar_images(bird_id, limit=5):
    """Search for images similar to a specific bird_id"""
    
    # Load features for reference
    with open('features/bird_image_features.pkl', 'rb') as f:
        features_list = pickle.load(f)
    
    # Connect to Qdrant
    client = QdrantClient(
        url=os.getenv('QDRANT_ENDPOINT'),
        api_key=os.getenv('QDRANT_API_KEY'),
    )
    
    # Find the target bird
    target_bird = None
    for item in features_list:
        if item['bird_id'] == bird_id:
            target_bird = item
            break
    
    if not target_bird:
        print(f"Bird ID {bird_id} not found")
        return
    
    # Search for similar images
    query_vector = target_bird['features'].tolist() if isinstance(target_bird['features'], np.ndarray) else target_bird['features']
    
    results = client.search(
        collection_name="bird_image_search",
        query_vector=query_vector,
        limit=limit
    )
    
    print(f"\nImages similar to bird_id {bird_id} ({target_bird['species_name']}):")
    for result in results:
        payload = result.payload
        print(f"  {payload['species_name']} (bird_id: {payload['bird_id']}, quality: {payload['quality_score']}, score: {result.score:.3f})")
